In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from finargy.api.client import InvertirOnlineAPI
import pandas as pd
import datetime
import numpy as np

In [ ]:
client = InvertirOnlineAPI()

In [ ]:
# print(client.access_token)

In [ ]:
historical_prices_path = "data/historical_prices.csv"
account_movements_path = "data/portfolio/account_movements_df.csv"

# Get historical prices
historical_prices = pd.read_csv(historical_prices_path)
historical_prices["fechaHora_date"] = pd.to_datetime(
    historical_prices["fechaHora_date"]
)
# get movements
movements_df = pd.read_csv(account_movements_path)
movements_df["fechaOperada"] = pd.to_datetime(movements_df["fechaOperada"])

In [ ]:
historical_prices.head()

In [ ]:
movements_df.columns

In [ ]:
# get discint types of movements
distinct_types = movements_df["tipo"].unique()
distinct_types

In [ ]:
# generate calendar_df

# convert to datetime
movements_df["fecha_operada"] = pd.to_datetime(movements_df["fecha_operada"])

start_date = movements_df["fecha_operada"].min()
end_date = pd.to_datetime(datetime.datetime.now().date())
calendar_df = pd.DataFrame(
    pd.date_range(start=start_date, end=end_date, freq="D"), columns=["fecha"]
)


# now generate this calendar for each unique asset in the movements_df using the min and max for each asset. If max is not available then propage until current date.

# get unique assets
assets = movements_df["simbolo"].unique()
assets_df = pd.DataFrame(assets, columns=["simbolo"])

# perform a cross join to generate calendar for each asset

df_combo = pd.merge(calendar_df, assets_df, how="cross")


# filter by min_date for each asset
min_date = (
    movements_df.groupby("simbolo")["fecha_operada"]
    .min()
    .reset_index()
    .rename(columns={"fecha_operada": "min_date"})
)

df_combo = pd.merge(df_combo, min_date, on="simbolo", how="left")

df_combo = df_combo[df_combo["fecha"] >= df_combo["min_date"]].reset_index(drop=True)

df_combo.rename(columns={"fecha": "fecha_operada"}, inplace=True)

In [ ]:
# read cedear ratio correction data

cedear_ratio_df = pd.read_csv(
    "data/portfolio/ajustes_cedears.csv",
    parse_dates=["fecha_operada"],
    date_format="%d/%m/%Y",
    usecols=["fecha_operada", "simbolo", "cantidad_correccion", "in_out", "market"],
)
cedear_ratio_df

# if out change sign
cedear_ratio_df["cantidad_correccion"] = np.where(
    cedear_ratio_df["in_out"] == "OUT",
    -cedear_ratio_df["cantidad_correccion"],
    cedear_ratio_df["cantidad_correccion"],
)

cedear_ratio_df

In [ ]:
# select relevant columns
movements_df = movements_df[
    [
        "tipo",
        "estado",
        "mercado",
        "simbolo",
        "cantidadOperada",
        "precioOperado",
        "montoOperado",
        "plazo",
        "fecha_orden",
        "fecha_operada",
    ]
].copy()

In [ ]:
# correct movements. If cedear ratio correction is available then add it to the movements_df
movements_df = pd.concat([movements_df, cedear_ratio_df], axis=0)



movements_df["cantidad_correccion"] = movements_df["cantidad_correccion"].fillna(0)



movements_df["cantidadOperada"] = movements_df["cantidadOperada"].fillna(0)



movements_df["cantidadOperada"] = (
    movements_df["cantidadOperada"] + movements_df["cantidad_correccion"]
)

In [ ]:
df_combo["fecha"] = pd.to_datetime(df_combo["fecha_operada"])
movements_df["fecha_operada"] = pd.to_datetime(movements_df["fecha_operada"])

# tipo == Venta then add - quantity
movements_df["cantidadOperada"] = np.where(
    movements_df["tipo"] == "Venta",
    -1 * movements_df["cantidadOperada"],
    movements_df["cantidadOperada"],
)


# Sort both dataframes by symbol and date
movements_df = movements_df.sort_values(["simbolo", "fecha_operada"]).reset_index(
    drop=True
)

df_combo = df_combo.sort_values(["simbolo", "fecha_operada"]).reset_index(drop=True)
# Calculate cumulative quantity (cum_qty)
movements_df["cum_qty"] = movements_df.groupby("simbolo")["cantidadOperada"].cumsum()

In [ ]:
movements_df[movements_df["simbolo"] == "VIST"]

In [ ]:
# Merge both dataframes using left join
df_merged = pd.merge(
    df_combo,
    movements_df,
    how="left",
    left_on=["simbolo", "fecha_operada"],
    right_on=["simbolo", "fecha_operada"],
)

In [ ]:
# perform forward fill "cantidadOperada" of for each asset
df_merged["total_qty"] = df_merged.groupby("simbolo")["cum_qty"].ffill()

In [ ]:
df_merged[df_merged["simbolo"] == "VIST"].reset_index().to_clipboard()

In [ ]:
# select relevant columns
df_merged = df_merged[
    [
        "fecha_operada",
        "cantidadOperada",
        "precioOperado",
        "montoOperado",
        "simbolo",
        "min_date",
        "market",
        "total_qty",
    ]
].copy()

In [ ]:
# get global min date of transactions

min_date = df_merged.groupby("simbolo")["fecha_operada"].min().min()
# convert to datetime.datetime

In [ ]:
from finargy.portfolio.analysis.mep import get_mep_dollar_history

dolar_mep = dolar_mep = get_mep_dollar_history(
    client,
    symbol="AL30",
    dollar_symbol="AL30D",
    date_start=min_date,
    date_end=datetime.datetime.now(),
)

In [ ]:
dolar_mep.head()

In [ ]:
# generate plot of mep dollar
import matplotlib.pyplot as plt

plt.plot(dolar_mep["fechaHora_date"], dolar_mep["dollar_mep"])
plt.show()

In [ ]:
df_prices = historical_prices.copy()

In [ ]:
df_merged[df_merged["simbolo"] == "VIST"].head()

In [ ]:
# join prices
df_merged_symbols = pd.merge(
    df_merged,
    df_prices,
    how="inner",
    left_on=["fecha_operada", "simbolo"],
    right_on=["fechaHora_date", "symbol"],
)

In [ ]:
# calculate total value
df_merged_symbols["total_value"] = (
    df_merged_symbols["total_qty"] * df_merged_symbols["ultimoPrecio"]
)

In [ ]:
# count total days since 2021-01-01

import datetime

start_date = datetime.datetime(2021, 1, 1)
end_date = datetime.datetime.now()
days = (end_date - start_date).days

# calculate total value in dollars
days

In [ ]:
# remove AL30D, GD30D, GD30, AL30 from the symbols
remove_symbols = ["AL30D", "GD30D", "GD30", "AL30"]
df_merged_symbols = df_merged_symbols[
    ~df_merged_symbols["simbolo"].isin(remove_symbols)
]

In [ ]:
df_merged_symbols.head()

In [ ]:
dolar_mep = dolar_mep[["fechaHora_date", "dollar_mep"]].copy()
# convert to date
dolar_mep["fechaHora_date"] = pd.to_datetime(dolar_mep["fechaHora_date"])

In [ ]:
dolar_mep.head()

In [ ]:
# join dolar mep
df_merged_symbols = pd.merge(
    df_merged_symbols,
    dolar_mep,
    how="left",
    on="fechaHora_date",
)

df_merged_symbols["total_value_usd"] = (
    df_merged_symbols["total_value"] / df_merged_symbols["dollar_mep"]
)

In [ ]:
# daily total_qty_variation
df_merged_symbols["total_qty_variation"] = df_merged_symbols["total_qty"].diff()

# fill first value with initial qty for each symbol

In [ ]:
# plot value of each symbol
df_merged_symbols["total_value_usd"] = df_merged_symbols["total_value_usd"].fillna(0)

import matplotlib.pyplot as plt
import seaborn as sns

# plot total value in dollars
plt.figure(figsize=(20, 10))
sns.lineplot(
    data=df_merged_symbols, x="fechaHora_date", y="total_value_usd", hue="simbolo"
)
plt.show()

In [ ]:
# max date by symbol
max_date = df_merged_symbols.groupby("simbolo")["fechaHora_date"].max()
max_date

In [ ]:
# get latest value for each symbol
latest_values = df_merged_symbols[
    df_merged_symbols["fechaHora_date"].isin(max_date)
].copy()

In [ ]:
latest_values.copy().to_clipboard()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the default style
plt.style.use("default")

# Create a figure and a set of subplots with secondary y-axis
fig, ax1 = plt.subplots()

# Plot the first dataset on the primary y-axis
ax1.plot(
    df_merged_VISTA["fechaHora_date"],
    df_merged_VISTA["total_value_usd"],
    label="Total Value USD",
)

ax1.set_xlabel("Date")
ax1.set_ylabel("Total Value (millions)")
ax1.tick_params(axis="x", rotation=45)

# Create a secondary y-axis
ax2 = ax1.twinx()

# Plot the second dataset on the secondary y-axis with increased bar width
ax2.bar(
    df_merged_VISTA["fechaHora_date"],
    df_merged_VISTA["cantidadOperada"],
    color="red",
    width=0.8,
    label="Total Qty Variation",
)
ax2.set_ylabel("Total Qty Variation")

# Add legends
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the default style
plt.style.use("default")

# Create a figure and a set of subplots with secondary y-axis
fig, ax1 = plt.subplots()

# Plot the first dataset on the primary y-axis
ax1.plot(
    df_merged_VISTA["fechaHora_date"],
    df_merged_VISTA["total_value"],
    label="Total Value USD",
)

ax1.set_xlabel("Date")
ax1.set_ylabel("Total Value (millions)")
ax1.tick_params(axis="x", rotation=45)

# Create a secondary y-axis
ax2 = ax1.twinx()

# Plot the second dataset on the secondary y-axis with increased bar width
ax2.bar(
    df_merged_VISTA["fechaHora_date"],
    df_merged_VISTA["cantidadOperada"],
    color="red",
    width=0.8,
    label="Total Qty Variation",
)
ax2.set_ylabel("Total Qty Variation")

# Add legends
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))

# Show the plot
plt.show()